# Import data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('/content/pca_results_5f_alldate.csv')

In [ ]:
data=data.drop(columns='Unnamed: 0')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
#Carol's path
data=pd.read_csv('/content/data.csv')
pca = pd.read_csv('/content/pca_results_7f_alldate.csv')

In [ ]:
data = pd.read_csv('data.csv')
data = data.set_index('DateTime')
data.tail()

,high,low,open,close,volumeto,ETH,SP500,Russell_2000,Crude_oil,Gold,Silver,EUR/USD,Treasury Yield 10 Years,USD/JPY,wiki_Bitcoin,wiki_Cryptocurrency,wiki_Coinbase
DateTime,,,,,,,,,,,,,,,,,
2021-11-26,59201.95,53575.54,58966.99,53788.22,3.312563e+09,4042.55,4594.620117,2245.939941,68.150002,1785.300049,23.107000,1.120963,1.482,115.069000,7057,5624,1720
2021-11-29,58898.57,56752.37,57331.41,57839.55,1.690006e+09,4448.49,4655.270020,2241.979980,69.949997,1782.300049,22.799000,1.129318,1.530,113.719002,7558,6341,2139
2021-11-30,59226.33,55931.33,57839.55,56975.35,2.383744e+09,4631.10,4567.000000,2198.909912,66.180000,1773.599976,22.764999,1.129344,1.443,113.804001,8005,6532,2477
2021-12-01,59096.53,56505.99,56975.35,57227.97,2.072824e+09,4587.74,4513.040039,2147.419922,65.570000,1781.599976,22.299000,1.133029,1.434,113.299004,6932,5674,2385
2021-12-02,57409.48,55850.40,57227.97,56455.09,1.332402e+09,4502.86,4583.049805,2190.657471,66.059998,1763.699951,22.350000,1.130199,1.458,113.162003,4597,3888,1229


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving pca_results_7f_alldate.csv to pca_results_7f_alldate.csv


In [ ]:
pca = pd.read_csv('pca_results_7f_alldate.csv')
pca = pca[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5', 'pc_6', 'pc_7']]
pca.index = data.index

In [ ]:
pca.tail()

,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7
DateTime,,,,,,,
2021-11-26,7.144434,0.364407,-2.157782,2.016871,0.113797,-1.153178,-0.451947
2021-11-29,7.140814,0.357431,-2.225540,1.843939,-0.139786,0.239506,-0.832773
2021-11-30,7.288559,0.426162,-1.827395,2.085112,0.172257,-0.169554,-0.617445
2021-12-01,7.047452,0.175963,-1.921008,1.968795,0.229543,0.003073,-0.600038
2021-12-02,6.540719,-0.416585,-2.696087,1.634378,-0.184203,0.037626,-0.963300


In [ ]:
# data[['pca_1', 'pca_2', 'pca_3', 'pca_4', 'pca_5']] = pca[['pc_ 1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]
data= pd.concat([data, pca], axis=1, join='inner')


In [ ]:
data['f_price']=data['close'].shift(-30)

In [ ]:
data['r']=data['close'].pct_change(30).shift(-30)
data['class']=np.where(data['r']>0,1,0)
# data['class']=np.where(data['r']<-0.1, -1, data['class'])

In [ ]:
data=data.dropna()

In [ ]:
data.tail()

,high,low,open,close,volumeto,ETH,SP500,Russell_2000,Crude_oil,Gold,Silver,EUR/USD,Treasury Yield 10 Years,USD/JPY,wiki_Bitcoin,wiki_Cryptocurrency,wiki_Coinbase,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,f_price,r,class
DateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-14,58514.76,56854.38,57369.00,57359.21,1.617497e+09,3791.23,4438.259766,2274.179932,81.309998,1796.699951,23.455999,1.159958,1.519,113.301003,8343,3895,1867,7.005658,0.285685,-2.559554,0.903543,-0.104890,0.481327,-0.675745,53788.22,-0.062257,0
2021-10-15,62889.98,56885.66,57359.21,61684.96,3.737890e+09,3868.16,4471.370117,2265.649902,82.279999,1767.199951,23.327999,1.159152,1.576,113.834000,9243,3710,1837,7.663466,0.665834,-2.376953,0.967492,0.615435,-0.987363,-0.411190,57839.55,-0.062340,0
2021-10-18,62662.72,59925.93,61520.54,62038.16,3.113486e+09,3746.14,4486.459961,2267.840088,82.440002,1764.800049,23.243000,1.159703,1.584,114.323997,9092,4335,1898,7.668160,0.742665,-2.490919,1.077909,0.378784,-0.594662,-0.462852,56975.35,-0.081608,0
2021-10-19,64468.70,61410.11,62038.16,64283.97,2.706420e+09,3877.45,4519.629883,2275.909912,82.959999,1769.699951,23.862000,1.161386,1.635,114.285004,8104,4537,2025,7.824595,0.711563,-2.615736,1.061750,0.299839,-0.245571,-0.512209,57227.97,-0.109763,0
2021-10-20,66980.66,63543.70,64283.97,66021.02,2.978059e+09,4162.49,4536.189941,2289.770020,83.870003,1784.099976,24.424000,1.163196,1.636,114.542999,9733,4721,2224,8.279484,0.868086,-2.558554,1.168953,0.375555,-0.311683,-0.551580,56455.09,-0.144892,0


In [ ]:
# Original Data
X = data[['high', 'low', 'open','close', 'volumeto', 'ETH', 'SP500', 'Russell_2000', 'Crude_oil', 'Gold', 'Silver', 'EUR/USD',
          'Treasury Yield 10 Years', 'USD/JPY', 'wiki_Bitcoin', 'wiki_Cryptocurrency', 'wiki_Coinbase'
]]

In [ ]:
# PCA with 5 features
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]

In [ ]:
# # PCA with 7 features
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5','pc_6', 'pc_7']]

In [ ]:
# make y as the 30 days future price
y_true = data['f_price']

# make y into 1 and 0. 1 means increase, 0 means decrease
y_tl = data['class']

# Run SVM

In [ ]:
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, random_state =22)
# x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, shuffle=False)


In [ ]:
# Try different kernels
from sklearn import svm
kernels = [ 'rbf', 'poly']
for kernel in kernels:
  svc = svm.SVC(kernel=kernel).fit(x_train, y_train)
  score = svc.score(x_test, y_test)
  print('kernels for', kernel, 'r2 socre is',score)

kernels for rbf r2 socre is 0.8066914498141264
kernels for poly r2 socre is 0.7546468401486989


In [ ]:
# Try different gamma for rbf
gammas = [0.1, 1, 10, 100]
for g in gammas:
   svc = svm.SVC(kernel='rbf', gamma=g, C= 100).fit(x_train, y_train)
   score = svc.score(x_test, y_test)
   print('gamma ==', g, 'r2 socre is',score)

gamma == 0.1 r2 socre is 0.8996282527881041
gamma == 1 r2 socre is 0.9144981412639405
gamma == 10 r2 socre is 0.8847583643122676
gamma == 100 r2 socre is 0.7657992565055762


In [ ]:
# Try different C
cs = [0.001,0.01, 0.1, 1, 10, 100]
for c in cs:
   svc = svm.SVC(kernel='rbf', C=c).fit(x_train, y_train)
   score = svc.score(x_test, y_test)
   print('C ==', c, 'r2 socre is',score)

C == 0.001 r2 socre is 0.6579925650557621
C == 0.01 r2 socre is 0.6579925650557621
C == 0.1 r2 socre is 0.7769516728624535
C == 1 r2 socre is 0.8066914498141264
C == 10 r2 socre is 0.8587360594795539
C == 100 r2 socre is 0.8884758364312267


In [ ]:
# Try different degrees
degrees = [0, 1, 2, 3, 4]
for degree in degrees:
   svc = svm.SVC(kernel='poly', degree=degree).fit(x_train, y_train)
   score = svc.score(x_test, y_test)
   print('degree ==', degree, 'r2 socre is',score)

degree == 0 r2 socre is 0.6579925650557621
degree == 1 r2 socre is 0.6691449814126395
degree == 2 r2 socre is 0.6914498141263941
degree == 3 r2 socre is 0.7546468401486989
degree == 4 r2 socre is 0.724907063197026


In [ ]:
svc = svm.SVC(kernel='rbf', C=100, gamma = 1).fit(x_train, y_train)
score = svc.score(x_test, y_test)
print('C ==100, gamma == 1', 'r2 socre is',score)


C ==100, gamma == 1 r2 socre is 0.9144981412639405


In [ ]:
# testing data
y_p2 = svc.predict(x_test)
y_p2

array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0])

In [ ]:
x_t = pca['2021-10-25':'2021-10-29'][['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]

In [ ]:
# This week's data
y_p2 = svc.predict(x_t)
y_pp=pd.DataFrame(y_p2, index= x_t.index, columns= ['y_svm'])


In [ ]:
y_pp

,y_svm
DateTime,
2021-10-25,0
2021-10-26,0
2021-10-27,1
2021-10-28,1
2021-10-29,0


# Neural Network

In [ ]:
# Import original data
X = data[['high', 'low', 'open','close', 'volumeto', 'ETH', 'SP500', 'Russell_2000', 'Crude_oil', 'Gold', 'Silver', 'EUR/USD',
          'Treasury Yield 10 Years', 'USD/JPY', 'wiki_Bitcoin', 'wiki_Cryptocurrency', 'wiki_Coinbase'
]]
y_tl = data['class']
# shuffle 0.3420
# non shuffle 0.2974

In [ ]:
# Import 5 features pca data
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]
y_tl = data['class']
# shuffle 0.8587
# non shuffle 0.3309

In [ ]:
# Import 7 features pca data
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5','pc_6', 'pc_7']]
y_tl = data['class']
# shuffle 0.7918
# non shuffle 0.3271

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, random_state =22)
# x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, shuffle=False)

In [ ]:
clf = MLPClassifier(solver='lbfgs',activation='relu', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [ ]:
# testing data
y_p3 = clf.predict(x_test)
y_p3

array([1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1])

In [ ]:
score = clf.score(x_test, y_test)
score

0.8587360594795539

In [ ]:
# This week's data
# x_t = pca['2021-10-18':'2021-10-22'][['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]
y_p3 = clf.predict(x_t)
# y_pp=pd.DataFrame(y_p, index= x_t.index)
y_pp['y_nn'] = y_p3
y_pp

,y_svm,y_nn
DateTime,,
2021-10-25,0,0
2021-10-26,0,0
2021-10-27,1,0
2021-10-28,1,1
2021-10-29,0,0


# Decision Tree

In [ ]:
# Import original data
X = data[['high', 'low', 'open','close', 'volumeto', 'ETH', 'SP500', 'Russell_2000', 'Crude_oil', 'Gold', 'Silver', 'EUR/USD',
          'Treasury Yield 10 Years', 'USD/JPY', 'wiki_Bitcoin', 'wiki_Cryptocurrency', 'wiki_Coinbase'
]]
y_tl = data['class']

In [ ]:
# Import 5 features pca data
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5']]
y_tl = data['class']

In [ ]:
# Import 7 features pca data
X = data[['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5','pc_6', 'pc_7']]
y_tl = data['class']

In [ ]:
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, random_state =22)
# x_train, x_test, y_train, y_test = train_test_split(X, y_tl, test_size = 0.2, shuffle=False)

In [ ]:
depth_list=list(range(1,11))
a_score=[]
print(depth_list)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
import matplotlib.pyplot as plt
from sklearn import tree
import graphviz

In [ ]:
for i in depth_list:
  treeClassifier=DecisionTreeClassifier(max_depth=i,min_samples_leaf=6)
  treeClassifier.fit(x_train,y_train)
  y_pred=treeClassifier.predict(x_test)
  a_score.append(accuracy_score(y_test, y_pred))

In [ ]:
plt.plot(depth_list,a_score)
plt.show

In [ ]:
a_score

In [ ]:
#when i =8 , the accuracy is the hightes so far, 
treeClassifier2=DecisionTreeClassifier(max_depth=8,min_samples_leaf=6)
treeClassifier2.fit(x_train,y_train)
y_p1 = treeClassifier2.predict(x_test)
y_p1
# y_pred2=treeClassifier2.predict(x_test)
# print((accuracy_score(y_test, y_pred2)))
# print(f1_score(y_test, y_pred2))
# print(precision_score(y_test, y_pred2))

In [ ]:
# This week's data
y_p1 = treeClassifier2.predict(x_t)
y_pp['y_dt'] = y_p1
y_pp

In [ ]:
treedata=tree.export_graphviz(treeClassifier2,filled=True, feature_names=['f1','f2','f3', 'f4', 'f5'],class_names=['0','1'])

In [ ]:
graphviz.Source(treedata)

In [ ]:
def dot_to_img(dot, file = 'decisiontreepca.png'):
    import os

    name, ext = os.path.splitext(file)

    graph = graphviz.Source(dot)
    graph.format = ext[1:]
    graph.view(name, cleanup = True) 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
dot_to_img(treedata,file = '/content/gdrive/My Drive/decisiontreepca.png')

# Check the result accuracy with our voting system
After we get the predict future return, we then calculate the difference between the original price and the returns to see whether the BTC price increases or decreases in 30 days.

In [ ]:
tmp = pd.DataFrame(y_test)
tmp

,class
DateTime,
2018-12-20,0
2018-10-03,0
2017-05-26,1
2020-12-28,1
2020-10-22,1
...,...
2018-10-02,0
2019-09-27,1
2020-01-03,1


In [ ]:
tmp['y_dt'] = y_p1
tmp.head()

In [ ]:
tmp['y_svm'] = y_p2
tmp.head()

,class,y_dt,y_svm
DateTime,,,
2018-12-20,0,0,0
2018-10-03,0,0,0
2017-05-26,1,1,1
2020-12-28,1,1,1
2020-10-22,1,1,1


In [ ]:
tmp['y_nn'] = y_p3
tmp.head()

,class,y_dt,y_svm,y_nn
DateTime,,,,
2018-12-20,0,0,0,1
2018-10-03,0,0,0,0
2017-05-26,1,1,1,1
2020-12-28,1,1,1,1
2020-10-22,1,1,1,1


In [ ]:
tmp['y_sum'] = tmp['y_dt'] + tmp['y_svm'] + tmp['y_nn']
tmp['y_pre'] = np.where(tmp['y_sum']>=2,1,0)
tmp.head()

,class,y_dt,y_svm,y_nn,y_sum,y_pre
DateTime,,,,,,
2018-12-20,0,0,0,1,1,0
2018-10-03,0,0,0,0,0,0
2017-05-26,1,1,1,1,3,1
2020-12-28,1,1,1,1,3,1
2020-10-22,1,1,1,1,3,1


In [ ]:
print( round(1 - sum(abs(tmp['y_pre'] - tmp['class']))/ tmp.shape[0], 4)*100, '%')

92.57 %


# Check Accuracy for this week

In [ ]:
import pandas as pd
# Download Bitcoin Trading data
import json
import requests

endpoint = 'https://min-api.cryptocompare.com/data/histoday'
res = requests.get(endpoint + '?fsym=BTC&tsym=USD&limit=2000')
BTC = pd.DataFrame(json.loads(res.content)['Data'])

# Use time as index
BTC = BTC.set_index('time')
BTC.index = pd.to_datetime(BTC.index, unit='s')
BTC = BTC[['close']]

In [ ]:
true_price = BTC.tail()

In [ ]:
true_price

,close
time,
2021-12-06,50552.83
2021-12-07,50633.01
2021-12-08,50513.04
2021-12-09,47593.85
2021-12-10,47425.62


In [ ]:
ndata = pd.read_csv('data.csv')
ndata = ndata.set_index('DateTime')
ndata.tail()
price_1mon_early= ndata['2021-10-25':'2021-10-29'][['close']]
p = price_1mon_early.iloc[:,0].values
p

array([63089.04, 60315.68, 58464.42, 60610.11, 62282.5 ])

In [ ]:
true_price['30daysbefore'] = p
true_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,close,30daysbefore
time,,
2021-12-06,50552.83,63089.04
2021-12-07,50633.01,60315.68
2021-12-08,50513.04,58464.42
2021-12-09,47593.85,60610.11
2021-12-10,47425.62,62282.50


In [ ]:
true_price['diff'] = true_price['close'] - true_price['30daysbefore']
true_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,close,30daysbefore,diff
time,,,
2021-12-06,50552.83,63089.04,-12536.21
2021-12-07,50633.01,60315.68,-9682.67
2021-12-08,50513.04,58464.42,-7951.38
2021-12-09,47593.85,60610.11,-13016.26
2021-12-10,47425.62,62282.50,-14856.88


In [ ]:
true_price['class'] = np.where(true_price['diff']>=0,1,0)
true_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,close,30daysbefore,diff,class
time,,,,
2021-12-06,50552.83,63089.04,-12536.21,0
2021-12-07,50633.01,60315.68,-9682.67,0
2021-12-08,50513.04,58464.42,-7951.38,0
2021-12-09,47593.85,60610.11,-13016.26,0
2021-12-10,47425.62,62282.50,-14856.88,0


In [ ]:
y_pp['sum'] = y_pp['y_dt'] + y_pp['y_svm'] + y_pp['y_nn']
y_pp['y_pre'] = np.where(y_pp['sum']>=2,1,0)
y_pp

,y_svm,y_nn,y_dt,sum,y_pre
DateTime,,,,,
2021-10-25,0,0,0,0,0
2021-10-26,0,0,0,0,0
2021-10-27,1,0,0,1,0
2021-10-28,1,1,0,2,1
2021-10-29,0,0,0,0,0
